In [1]:
# Import the dependencies.
import pandas as pd
from citipy import citipy
import numpy as np
import os
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

630

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key + "&lang=en"

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
   
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        WeatherDescriptionList  = city_weather["weather"]
        WeatherDescription = WeatherDescriptionList[0]["description"]
        try:
            Rain1HR = "{:.1f}".format(city_weather["rain"]["1h"] / 25.4)
        except:
            Rain1HR = "{:.1f}".format(0)
        try:
            Snow1HR = "{:.1f}".format(city_weather["snow"]["1h"] / 25.4)
        except:
            Snow1HR = "{:.1f}".format(0)
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description" : WeatherDescription,
                          "Rain (Inches)" : Rain1HR,
                          "Snow (Inches)" : Snow1HR,
                          "Date" : city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

rto ayora
Processing Record 24 of Set 4 | luderitz
Processing Record 25 of Set 4 | seydi
Processing Record 26 of Set 4 | srednekolymsk
Processing Record 27 of Set 4 | tasiilaq
Processing Record 28 of Set 4 | geraldton
Processing Record 29 of Set 4 | portland
Processing Record 30 of Set 4 | aksarka
Processing Record 31 of Set 4 | estelle
Processing Record 32 of Set 4 | trincomalee
Processing Record 33 of Set 4 | qinzhou
Processing Record 34 of Set 4 | benghazi
Processing Record 35 of Set 4 | korla
Processing Record 36 of Set 4 | lorengau
Processing Record 37 of Set 4 | manching
Processing Record 38 of Set 4 | arandis
Processing Record 39 of Set 4 | codrington
Processing Record 40 of Set 4 | port hedland
Processing Record 41 of Set 4 | strezhevoy
Processing Record 42 of Set 4 | brahmapuri
City not found. Skipping...
Processing Record 43 of Set 4 | alofi
Processing Record 44 of Set 4 | santa barbara
Processing Record 45 of Set 4 | kabare
Processing Record 46 of Set 4 | broken hill
Process

In [7]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches),Date
0,Bonfim,BR,3.08,-59.95,87.37,61,96,4.90,light rain,0.0,0.0,2020-07-09 20:21:58
1,Victoria,HK,22.29,114.16,86.00,74,100,11.01,overcast clouds,0.0,0.0,2020-07-09 20:21:58
2,Cape Town,ZA,-33.93,18.42,55.00,93,75,6.93,moderate rain,0.1,0.0,2020-07-09 20:21:13
3,Manicore,BR,-5.81,-61.30,89.60,55,20,5.82,few clouds,0.0,0.0,2020-07-09 20:21:58
4,Hobart,AU,-42.88,147.33,37.40,93,20,6.93,light snow,0.0,0.0,2020-07-09 20:18:13
5,Ushuaia,AR,-54.80,-68.30,37.40,64,40,5.82,scattered clouds,0.0,0.0,2020-07-09 20:21:59
6,Aloleng,PH,16.13,119.78,82.74,80,99,7.11,light rain,0.0,0.0,2020-07-09 20:21:59
7,Iracoubo,GF,5.48,-53.20,81.82,78,35,6.20,light rain,0.0,0.0,2020-07-09 20:21:59
8,Porbandar,IN,21.64,69.61,84.00,82,31,19.44,light rain,0.0,0.0,2020-07-09 20:21:59
9,Makakilo City,US,21.35,-158.09,84.20,58,75,9.17,broken clouds,0.0,0.0,2020-07-09 20:21:59


In [8]:
# Create the output file (CSV).
output_data_file = os.path.join("data","WeatherPy_database.csv")
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")